In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [16]:
from IPython.display import Javascript, HTML, Markdown, display
from ipywidgets import widgets
import numpy as np
import re
import nltk
import pandas as pd

def printmd(string):
    display(Markdown(string))
    
printmd("Falls nach einigen Sekunden nichts angezeigt wird, bitte die Schaltfl&auml;che bet&auml;tigen:")

def run_script(ev):
    display(Javascript("IPython.notebook.execute_cells_below()"))
button = widgets.Button(description="Skript ausführen")
button.on_click(run_script)
display(button)

Falls nach einigen Sekunden nichts angezeigt wird, bitte die Schaltfl&auml;che bet&auml;tigen:

Button(description='Skript ausführen', style=ButtonStyle())

In [17]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()"><b>Gesamten Quellcode ein-/ausblenden</b></a>.''')

In [18]:
df1 = pd.read_json("amazon/misc_reviews.json")
df2 = pd.read_json("amazon/pommersche.json")
df3 = pd.read_json("amazon/more_reviews.json")
df4 = pd.read_json("vantastic_foods/vantastic_fleischersatz.json")
df5 = pd.read_json("vantastic_foods/vantastic_misc.json")
df6 = pd.read_json("vekoop/reviews.json")

df6 = df6.rename(columns={"text": "review", "date_time": "date", "topic_title": "product_name"})
dfa = pd.concat([df1, df2, df3])
dfb = pd.concat([df4, df5])

dfa = dfa.drop_duplicates()
dfb = dfb.drop_duplicates()
df6 = df6.drop_duplicates()

dfa["date"] = dfa["date"].replace("(Rezension aus Deutschland vom )", "", regex=True)
dfa["date"] = dfa["date"].replace({"Januar": "Jan", "Februar": "Feb", "März": "Mar", "April": "Apr", "Mai": "May", "Juni": "Jun", "Juli": "Jul", "August": "Aug", "Semptember": "Sep", "Oktober": "Oct", "November": "Nov", "Dezember": "Dec"}, regex=True)
dfa = dfa[~dfa.date.str.contains("Königreich")]
dfa.date = pd.to_datetime(dfa.date)

In [19]:
df = pd.concat([dfa, dfb, df6])
df = df.reset_index(drop=True)
df = df[df["review"].notna()]
printmd("<b>Struktur des vollst&auml;ndigen Datensatzes</b>")
df.info()

<b>Struktur des vollst&auml;ndigen Datensatzes</b>

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 5107
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   product_name  5100 non-null   object        
 1   author        5098 non-null   object        
 2   rating        5100 non-null   int64         
 3   title         5100 non-null   object        
 4   date          5100 non-null   datetime64[ns]
 5   review        5100 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 278.9+ KB


In [20]:
df["positive"] = np.where(df["rating"] > 3, 1, 0)

df_pos_temp = df[["review", "title"]][df.positive == 1]
df_neg_temp = df[["review", "title"]][df.positive == 0]

series_pos = pd.concat([df_pos_temp.title, df_pos_temp.review])
series_neg = pd.concat([df_neg_temp.title, df_neg_temp.review])

df_pos = series_pos.to_frame()
df_neg = series_neg.to_frame()
df_pos["positive"] = 1
df_pos = df_pos.rename(columns={0: "text"})
df_neg["positive"] = 0
df_neg = df_neg.rename(columns={0: "text"})
df_samples = pd.concat([df_pos, df_neg])
df_samples = df_samples.reset_index(drop=True)

df_backup = df_samples
printmd("<b>Struktur des Text-Datensatzes</b>")
df_samples.info()

<b>Struktur des Text-Datensatzes</b>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      10200 non-null  object
 1   positive  10200 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 159.5+ KB


In [21]:
HTML('''<script>
function toggle_next(event) {
    $(event.target).closest('div.code_cell').next().find('div.output').toggle();
}
</script>
<a onclick="toggle_next(event)"><b>Stopp-W&ouml;rter ein-/ausblenden</b></a>''')

In [22]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from urllib.request import urlopen

stemmer = SnowballStemmer("german")
with urlopen("https://raw.githubusercontent.com/gosia-malgosia/german-stop-words/master/german-stop-words.txt") as file:
    stopwords_encoded = file.read().splitlines()
#stopwords = [word.decode('utf8') for word in stopwords_encoded]
#with open("de_stopwords.json", encoding="utf-8") as stopw:
#    stopwords = stopw.read()
stopwords = set(stopwords.words("german"))

def text_filter(words):
    re_wspace = re.compile(r"\s+", re.IGNORECASE)
    re_ascii = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    re_singlechar = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    words = re.sub(re_wspace, " ", words)
    words = re.sub(re_ascii, " ", words)
    words = re.sub(re_singlechar, " ", words)
    tokenized = word_tokenize(words)
    text_lower = [token.lower() for token in tokenized]
    stopwords_removed = [word for word in text_lower if word not in stopwords]
    stemmed_words = [stemmer.stem(word) for word in stopwords_removed]
    filtered = " ".join(stemmed_words)
    return filtered

def filter_nostemming(words):
    re_wspace = re.compile(r"\s+", re.IGNORECASE)
    re_ascii = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    re_singlechar = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    words = re.sub(re_wspace, " ", words)
    words = re.sub(re_ascii, " ", words)
    words = re.sub(re_singlechar, " ", words)
    tokenized = word_tokenize(words)
    text_lower = [token.lower() for token in tokenized]
    stopwords_removed = [word for word in text_lower if word not in stopwords]
    
    filtered_notstemmed = " ".join(stopwords_removed)
    return filtered_notstemmed

print(stopwords)

{'einiger', 'und', 'anderem', 'habe', 'meine', 'selbst', 'meinen', 'eure', 'aus', 'deinem', 'bis', 'desselben', 'einer', 'sondern', 'denselben', 'aller', 'ihm', 'ihren', 'von', 'eurem', 'so', 'einige', 'derselbe', 'ihn', 'dazu', 'jenen', 'nur', 'deinen', 'sein', 'meines', 'zum', 'hier', 'dem', 'solchem', 'weil', 'jedem', 'keine', 'könnte', 'allem', 'jedes', 'demselben', 'das', 'der', 'deines', 'uns', 'einmal', 'welches', 'diesen', 'solche', 'bei', 'ihnen', 'anderm', 'zwar', 'diese', 'manche', 'zwischen', 'daß', 'euer', 'andere', 'des', 'kann', 'einig', 'sind', 'über', 'oder', 'eine', 'warst', 'dir', 'dieselbe', 'dieses', 'keines', 'machen', 'sonst', 'du', 'gewesen', 'kein', 'welchem', 'will', 'vom', 'denn', 'aber', 'hat', 'ihrem', 'haben', 'muss', 'hinter', 'keinem', 'alle', 'weiter', 'dort', 'seine', 'was', 'als', 'zu', 'mich', 'jeden', 'keiner', 'seiner', 'alles', 'im', 'jetzt', 'anders', 'er', 'keinen', 'wollen', 'vor', 'doch', 'welchen', 'da', 'indem', 'meiner', 'jenes', 'auf', 'so

In [23]:
df_samples["clean_text"] = df_samples["text"].apply(lambda row: text_filter(row))
df_backup["clean_text"] = df_backup["text"].apply(lambda row: filter_nostemming(row))
df_samples.clean_text.str.split(expand=True).stack().value_counts()
printmd("Originaltext <b>&quot;text&quot;</b> vs normalisierter Text <b>&quot;clean_text&quot;</b>")
#df_samples.head()

df_samples.loc[[5600,5601,5602,5603,5604,5605,5606,5607,5608,5609],:]

Originaltext <b>&quot;text&quot;</b> vs normalisierter Text <b>&quot;clean_text&quot;</b>

,text,positive,clean_text
5600,Endlich ein verdammt leckere Alternative! Supe...,1,endlich verdammt leckere alternative super tol...
5601,sehr leckerer Leberwurstgeschmack nach meinem ...,1,leckerer leberwurstgeschmack empfinden konsist...
5602,Bisher wurde ich immer enttäuscht von dem Gesc...,1,bisher wurde immer enttäuscht geschmack vegane...
5603,Dieser Aufstrich gehört bei mir zum Frühstück ...,1,aufstrich gehört frühstück abendbrot schon sta...
5604,von konsistenz und geschmack kaum von echter l...,1,konsistenz geschmack kaum echter leberwurst un...
5605,War leider schon drei Tage nachdem ich das Pak...,1,leider schon drei tage nachdem paket bekommen ...
5606,Diese Violife-Scheiben schmecken mir gut. Ich ...,1,violife scheiben schmecken gut weiß jedoch piz...
5607,"Selbst mein Omni hat die Scheiben gern, weil d...",1,omni scheiben gern leckere pizza geschmack ebe...
5608,"Angenehm frisch, leicht süßlich, mindestens so...",1,angenehm frisch leicht süßlich mindestens leck...
5609,"Wirklich köstlich, ich bin begeistert. Schade,...",1,wirklich köstlich begeistert schade wenig drin


In [24]:
import hvplot
import holoviews as hv
from hvplot import pandas
from bokeh.models import NumeralTickFormatter

word_freq = pd.Series(" ".join(df_backup["clean_text"]).split()).value_counts()
word_freq[1:40].rename("Am häufigsten verwendete Wörter im Datensatz").hvplot.bar(
    rot=45
).opts(width=700, height=400, yformatter=NumeralTickFormatter(format="0,0"))

:Bars   [index]   (Am häufigsten verwendete Wörter im Datensatz)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3)
vectorizer.fit(df_samples["clean_text"])

train, test = train_test_split(df_samples, random_state=1, test_size=0.1)
x_train = train["clean_text"]
y_train = train["positive"]
x_test = test["clean_text"]
y_test = test["positive"]

training_scores = vectorizer.transform(x_train)
test_scores = vectorizer.transform(x_test)

print("Training Sample Size: {}".format(x_train.shape[0]))
print("Test Sample Size: {}".format(x_test.shape[0]))

Training Sample Size: 9180
Test Sample Size: 1020


In [26]:
classifier = MultinomialNB()
classifier.fit(training_scores, y_train)
prediction = classifier.predict(test_scores)
printmd("Aktuelle G&uuml;te des Modells:\n")
print(classification_report(y_test, prediction))


Aktuelle G&uuml;te des Modells:


              precision    recall  f1-score   support

           0       0.93      0.33      0.48       196
           1       0.86      0.99      0.92       824

    accuracy                           0.87      1020
   macro avg       0.89      0.66      0.70      1020
weighted avg       0.87      0.87      0.84      1020



In [27]:
pipeline = make_pipeline(TfidfVectorizer(ngram_range=(1, 3), min_df=6), MultinomialNB())
pipeline.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(min_df=6, ngram_range=(1, 3))),
                ('multinomialnb', MultinomialNB())])

In [28]:
printmd("<h3>Bitte hier Text eingeben und mit &quot;Enter&quot; best&auml;tigen:</h3>")
text = widgets.Text()
display(text)

def get_submit(mock_review):
    text = text_filter(mock_review.value)
    prediction = pipeline.predict([text])
    print("\"{}\"".format(mock_review.value))
    if mock_review == None:
        pass
    elif prediction == 0:
        print("negativ")
        print(pipeline.predict_proba([text]))
        
    elif prediction == 1:
        print("positiv")
        print(pipeline.predict_proba([text]))
        
    else:
        print("Fehler")
    print("\n")

text.on_submit(get_submit)

<h3>Bitte hier Text eingeben und mit &quot;Enter&quot; best&auml;tigen:</h3>

Text(value='')

In [29]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()"><b>Gesamten Quellcode ein-/ausblenden</b></a>.''')

In [31]:
printmd("--- end of notebook ---")

--- end of notebook ---